In [2]:
import pandas as pd

df1 = pd.DataFrame(columns=["Date", "Time", "Title"])
df1

,Date,Time,Title


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd
from datetime import datetime, timedelta

def get_df(df_res, driver):
    # 重新查找所有包含节目单的 <p> 元素
    program_elements = driver.find_elements(By.CSS_SELECTOR, 'div#liveepg p.normal')
    program_time = []
    program_name = []
    program_date = []
    # 遍历并提取每个节目的日期、时间和名称
    for program in program_elements:
        # 尝试重新获取节目时间和名称
        try:
            # 获取节目时间
            time_element = program.find_element(By.CSS_SELECTOR, 'span.epgtime')
            program_time.append(time_element.text)
            
            # 获取节目名称
            name_element = program.find_elements(By.TAG_NAME, 'span')[1]
            program_name.append(name_element.text)
    
            # 获取节目日期 (从title属性中提取日期信息)
            program_date.append(program.get_attribute('title').split(' ')[0])
    
            # 打印日期、时间和节目名称
            # print(f"日期: {program_date}, 时间: {program_time}, 节目: {program_name}")
        except StaleElementReferenceException:
            print("元素已过期，需要重新加载页面元素")
            
    data = {
        'program_date': program_date,
        'program_time': program_time,
        'program_name': program_name
    }
    
    # 创建 DataFrame
    df = pd.DataFrame(data)

    # 将 program_date 转换为 datetime 格式
    df['program_date'] = pd.to_datetime(df['program_date'], format='%m月%d日')

    # 创建新的 df_new DataFrame
    df_new = pd.DataFrame()

    # 填充 df_new
    df_new['Date'] = df['program_date'].apply(lambda x: x.replace(year=2024))
    df_new['Time'] = pd.to_datetime(df_new['Date'].dt.strftime('%Y-%m-%d') + ' ' + df['program_time'])
    # df_new['end'] = df_new['start'] + timedelta(minutes=80)
    df_new['Date'] = df_new['Date'].dt.strftime('%Y-%m-%d')
    df_new['Time'] = df_new['Time'].dt.strftime('%H:%M:%S')
    # df_new['end'] = df_new['end'].dt.strftime('%H:%M:%S')
    df_new['Title'] = df['program_name']

    return pd.concat([df_res, df_new])

In [6]:
def get_data_for_week(driver, df):
    for i in range(7):
        next_week_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, f"weekday_{i}"))
        )
        next_week_button.click()
        time.sleep(1)
        df = get_df(df, driver)
    return df
    
# 设置ChromeDriver路径
driver_path = r'C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe'  # 替换为你下载的ChromeDriver路径
service = Service(driver_path)

# 启动Chrome浏览器
driver = webdriver.Chrome(service=service)

# 打开目标网址
url = 'http://epg.tv.cn/soccer'  # 替换为实际节目单的URL
driver.get(url)

# 等待页面完全加载
time.sleep(1)
df2 = get_df(df1, driver)

## Current week
# next_week_button = WebDriverWait(driver, 10).until(
#     EC.element_to_be_clickable((By.ID, "weekday_0"))
# )
# next_week_button.click()
# time.sleep(1)
df2 = get_data_for_week(driver, df2)
# df2 = get_df(df2, driver)

## Last week
next_week_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "upweek"))
)
next_week_button.click()
time.sleep(1)



df2 = get_data_for_week(driver, df2)

'''next_week_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "upweek"))
)
next_week_button.click()
time.sleep(1)

for i in range(7):
    next_week_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, f"weekday_{i}"))
    )
    next_week_button.click()
    time.sleep(1)
    df2 = get_df(df2, driver)'''
    
# 关闭浏览器
driver.quit()


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 128
Current browser version is 130.0.6723.69 with binary path C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
Stacktrace:
	GetHandleVerifier [0x00007FF7AFAFB5D2+29090]
	(No symbol) [0x00007FF7AFA6E689]
	(No symbol) [0x00007FF7AF92B1CA]
	(No symbol) [0x00007FF7AF96D6B2]
	(No symbol) [0x00007FF7AF96C751]
	(No symbol) [0x00007FF7AF96729C]
	(No symbol) [0x00007FF7AF961E0A]
	(No symbol) [0x00007FF7AF9B236D]
	(No symbol) [0x00007FF7AF9B1A20]
	(No symbol) [0x00007FF7AF9A6493]
	(No symbol) [0x00007FF7AF9709B1]
	(No symbol) [0x00007FF7AF971B11]
	GetHandleVerifier [0x00007FF7AFE18C5D+3295277]
	GetHandleVerifier [0x00007FF7AFE64843+3605523]
	GetHandleVerifier [0x00007FF7AFE5A707+3564247]
	GetHandleVerifier [0x00007FF7AFBB6EB6+797318]
	(No symbol) [0x00007FF7AFA7980F]
	(No symbol) [0x00007FF7AFA753F4]
	(No symbol) [0x00007FF7AFA75580]
	(No symbol) [0x00007FF7AFA64A1F]
	Ba

In [111]:
df2.index = range(len(df2))

In [113]:
df2['Date'] = pd.to_datetime(df2['Date'])

In [120]:
df_final = df2[ (df2['Date'] <= pd.to_datetime('2024-10-22')) & (df2['Date'] >= pd.to_datetime('2024-10-15')) ].sort_values(by=['Date', 'Time'])

In [122]:
df_final.to_excel("./tmp.xlsx", index=False)